Import Library

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Activation, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import cv2

Define the dataset paths and parameters

In [ ]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
base_dir = '/content/drive/MyDrive/Acne Detection Model/merged_dataset'
train_data_dir = os.path.join(base_dir, 'train', 'dataset')
val_data_dir = os.path.join(base_dir, 'valid', 'dataset')
test_data_dir = os.path.join(base_dir, 'test', 'dataset')
annotations_train_file = os.path.join(base_dir, 'train', '_annotations.csv')
annotations_val_file = os.path.join(base_dir, 'valid', '_annotations.csv')
annotations_test_file = os.path.join(base_dir, 'test', '_annotations.csv')
annotations_train = pd.read_csv(annotations_train_file)
annotations_val = pd.read_csv(annotations_val_file)

Load and preprocess the data

In [ ]:


# Create image data generators with augmentation for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.5,
    horizontal_flip=True,
    vertical_flip=True
)

# Create image data generators without augmentation for training and validation
""" train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
 """
# Load and preprocess the training data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=annotations_train,
    directory=train_data_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Create image data generators without augmentation for training and validation
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and preprocess the validation data
val_generator = val_datagen.flow_from_dataframe(
    dataframe=annotations_val,
    directory=val_data_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 41040 validated image filenames belonging to 4 classes.
Found 2881 validated image filenames belonging to 4 classes.


In [ ]:
batch_size = 32
num_classes = annotations_train['class'].nunique()

Build the model

In [ ]:
# Load the MobileNetV2 model (pretrained on ImageNet)
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the initial layers
for layer in base_model.layers:
    layer.trainable = False
    
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer=l1(0.01))(x)  # Add L1 regularization
x = BatchNormalization()(x)  # Add batch normalization
x = Dropout(0.5)(x)  # Add dropout after the first hidden layer
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  # Add L2 regularization
x = BatchNormalization()(x)  # Add batch normalization
x = Dropout(0.5)(x)  # Add dropout after the second hidden layer
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


Fine Tuning

In [ ]:
# Setting the top layer as trainable
base_model.trainable = True

In [ ]:
# Perform fine-tuning
# Unfreeze the last few layers of the base model
# Let's take a look to see how many layers are in the base model
print('Number of layers in the base model: ', len(base_model.layers))

# Fine-tune from this layer onwards 
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
# Re-compile the model to apply the changes
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

Number of layers in the base model:  154


In [ ]:
# Printing the model architecture summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
# Printing the no. of trainable_variables of the model
len(model.trainable_variables)

64

Train the model

In [ ]:
# Train the model and store the training metrics
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
)

Epoch 1/10
  14/1283 [..............................] - ETA: 3:42:24 - loss: 1.5861 - accuracy: 0.3817

KeyboardInterrupt: ignored

Visualize

In [ ]:

# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

Evaluate the model

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load the annotations test file into a DataFrame
df = pd.read_csv(annotations_test_file)
# Load and preprocess the test data
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory=test_data_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)

print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')


Save Model

In [ ]:
model_path = ('/content/drive/MyDrive/Acne Detection Model/Demo/model_mobilenetv2_V2.h5')
model.save(model_path)
#model.save_weights(model_path)